In [1]:
import torch
import torch.nn as nn

from torch_geometric.nn.pool import global_mean_pool
from torch.utils.data import ConcatDataset, DataLoader, random_split, Sampler

In [2]:
from models_by_hand import GCNConvByHand
from helpers import CVFConfigForGCNWSuccWEIDataset

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [4]:
dataset_s_n7 = CVFConfigForGCNWSuccWEIDataset(
    device,
    "star_graph_n7_config_rank_dataset.csv",
    "star_graph_n7_edge_index.json",
)

dataset_s_n13 = CVFConfigForGCNWSuccWEIDataset(
    device,
    "star_graph_n13_config_rank_dataset.csv",
    "star_graph_n13_edge_index.json",
)

dataset_rr_n7 = CVFConfigForGCNWSuccWEIDataset(
    device,
    "graph_random_regular_graph_n7_d4_config_rank_dataset.csv",
    "graph_random_regular_graph_n7_d4_edge_index.json",
)

dataset_plc_n7 = CVFConfigForGCNWSuccWEIDataset(
    device,
    "graph_powerlaw_cluster_graph_n7_config_rank_dataset.csv",
    "graph_powerlaw_cluster_graph_n7_edge_index.json",
)


dataset_implict_n5 = CVFConfigForGCNWSuccWEIDataset(
    device,
    "implicit_graph_n5_config_rank_dataset.csv",
    "implicit_graph_n5_edge_index.json",
    "dijkstra",
)

dataset_implict_n10 = CVFConfigForGCNWSuccWEIDataset(
    device,
    "implicit_graph_n10_config_rank_dataset.csv",
    "implicit_graph_n10_edge_index.json",
    "dijkstra",
)

dataset_implict_n7 = CVFConfigForGCNWSuccWEIDataset(
    device,
    "implicit_graph_n7_config_rank_dataset.csv",
    "implicit_graph_n7_edge_index.json",
    "dijkstra",
)

batch_size = 64

dataset_coll = [
    # dataset_implict_n5,
    # dataset_implict_n10,
    # dataset_implict_n7,
    dataset_s_n7,
    # dataset_s_n13,
    dataset_rr_n7,
    dataset_plc_n7
]

In [5]:
train_sizes = [int(0.95 * len(ds)) for ds in dataset_coll]
test_sizes = [len(ds) - trs for ds, trs in zip(dataset_coll, train_sizes)]

train_test_datasets = [
    random_split(ds, [tr_s, ts])
    for ds, tr_s, ts in zip(dataset_coll, train_sizes, test_sizes)
]

train_datasets = [ds[0] for ds in train_test_datasets]
test_datasets = [ds[1] for ds in train_test_datasets]

In [6]:
datasets = ConcatDataset(train_datasets)
print(f"Train Dataset size: {len(datasets):,}")

Train Dataset size: 100,293


In [7]:
class CustomBatchSampler(Sampler):
    def __init__(self, datasets: ConcatDataset, batch_size: int):
        self.datasets = datasets
        self.batch_size = batch_size

    def __iter__(self):
        last_accessed = [0] + self.datasets.cumulative_sizes[:]
        end_loop = [False for _ in range(len(self.datasets.datasets))]

        while not all(end_loop):
            for turn in range(len(self.datasets.datasets)):
                if end_loop[turn]:
                    continue

                batch_size = self.batch_size
                if (
                    last_accessed[turn] + batch_size
                    >= self.datasets.cumulative_sizes[turn]
                ):
                    batch_size = (
                        self.datasets.cumulative_sizes[turn] - last_accessed[turn]
                    )
                    end_loop[turn] = True

                yield list(range(last_accessed[turn], last_accessed[turn] + batch_size))

                last_accessed[turn] += batch_size

In [8]:
batch_sampler = CustomBatchSampler(datasets, batch_size=batch_size)
dataloader = DataLoader(datasets, batch_sampler=batch_sampler)

In [9]:
class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        # self.gcn = GCNConvByHand(input_size, input_size, bias=False, device=device)
        self.rnn = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.h2o = nn.Linear(hidden_size, output_size)

    def forward(self, x, A):
        # h = self.gcn(x, A)
        # h = torch.relu(h)
        rnn_out, _ = self.rnn(x)
        output = self.h2o(rnn_out)
        output = torch.relu(output)
        output = global_mean_pool(output, torch.zeros(output.size(1)).to(device).long())
        return output

    def fit(self, epochs):
        criterion = torch.nn.MSELoss()
        optimizer = torch.optim.Adam(self.parameters(), lr=0.01, weight_decay=0.0001)
        for epoch in range(1, epochs + 1):
            self.train()
            total_loss = 0
            count = 0
            for batch in dataloader:
                x = batch[0]
                y = batch[1]
                y = y.unsqueeze(-1)
                out = self(x[0], x[1])
                # print(out.shape, y.shape)
                optimizer.zero_grad()
                loss = criterion(out, y)
                total_loss += loss
                count += 1
                loss.backward()
                optimizer.step()

            print(
                "Training set | Epoch",
                epoch,
                "| MSE Loss:",
                round((total_loss / count).item(), 4),
            )

In [10]:
D = 3
H = 16

model = SimpleRNN(D, H, 1).to(device)
print(model)
print()
print("Total parameters:", f"{sum(p.numel() for p in model.parameters()):,}")
print()
model.fit(epochs=20)

SimpleRNN(
  (rnn): LSTM(3, 16, batch_first=True)
  (h2o): Linear(in_features=16, out_features=1, bias=True)
)

Total parameters: 1,361

Training set | Epoch 1 | MSE Loss: 0.406
Training set | Epoch 2 | MSE Loss: 0.0936
Training set | Epoch 3 | MSE Loss: 0.0825
Training set | Epoch 4 | MSE Loss: 0.0734
Training set | Epoch 5 | MSE Loss: 0.0687
Training set | Epoch 6 | MSE Loss: 0.0659
Training set | Epoch 7 | MSE Loss: 0.0626
Training set | Epoch 8 | MSE Loss: 0.059
Training set | Epoch 9 | MSE Loss: 0.056
Training set | Epoch 10 | MSE Loss: 0.0538
Training set | Epoch 11 | MSE Loss: 0.0522
Training set | Epoch 12 | MSE Loss: 0.051
Training set | Epoch 13 | MSE Loss: 0.0497
Training set | Epoch 14 | MSE Loss: 0.0485
Training set | Epoch 15 | MSE Loss: 0.0468
Training set | Epoch 16 | MSE Loss: 0.045
Training set | Epoch 17 | MSE Loss: 0.0434
Training set | Epoch 18 | MSE Loss: 0.0423
Training set | Epoch 19 | MSE Loss: 0.0414
Training set | Epoch 20 | MSE Loss: 0.0407


# Testing

In [11]:
import csv
import datetime

f = open(f"test_results/test_result_w_succ_diff_nodes_lstm_{datetime.datetime.now().strftime("%Y_%m_%d_%H_%M")}.csv", "w", newline="")
csv_writer = csv.writer(f)
csv_writer.writerow(["Dataset", "Actual", "Predicted"])

criterion = torch.nn.MSELoss()

model.eval()

with torch.no_grad():
    test_concat_datasets = ConcatDataset(test_datasets)
    test_batch_sampler = CustomBatchSampler(test_concat_datasets, batch_size=10240)
    test_dataloader = DataLoader(test_concat_datasets, batch_sampler=test_batch_sampler)

    total_loss = 0
    total_matched = 0
    count = 0
    for batch in test_dataloader:
        x = batch[0]
        y = batch[1]
        y = y.unsqueeze(-1)
        out = model(x[0], x[1])
        csv_writer.writerows(
            (i, j.item(), k.item())
            for (i, j, k) in zip(
                x[2], y.detach().cpu().numpy(), out.detach().cpu().numpy()
            )
        )
        loss = criterion(out, y)
        total_loss += loss
        out = torch.round(out)
        matched = (out == y).sum().item()
        total_matched += matched
        count += 1

    print(
        "Test set",
        "| MSE loss:",
        round((total_loss / count).item(), 4),
        "| Total matched",
        total_matched,
        "out of",
        len(test_concat_datasets),
        f"(Accuracy: {round(total_matched/len(test_concat_datasets) * 100, 2)}%)",
    )

f.close()

Test set | MSE loss: 0.0394 | Total matched 5124 out of 5280 (Accuracy: 97.05%)


# Testing with Untrained Datasets

In [13]:
dataset = dataset_s_n13

with torch.no_grad():
    test_dataloader = DataLoader(dataset, batch_size=10240)
    total_loss = 0
    total_matched = 0
    count = 0
    for batch in test_dataloader:
        x = batch[0]
        y = batch[1]
        y = y.unsqueeze(-1)
        out = model(x[0], x[1])
        loss = criterion(out, y)
        total_loss += loss
        out = torch.round(out)
        matched = (out == y).sum().item()
        total_matched += matched
        count += 1

    print(
        "Test set",
        f"| {dataset.dataset_name}",
        "| MSE loss:",
        round((total_loss / count).item(), 4),
        "| Total matched",
        f"{total_matched:,}",
        "out of",
        f"{len(dataset):,}",
        f"(Accuracy: {round(total_matched/len(dataset) * 100, 2)}%)",
    )

Test set | star_graph_n13 | MSE loss: 0.1201 | Total matched 48,142 out of 53,248 (Accuracy: 90.41%)
